In [1]:
# project 4 notebook a

In [2]:
#q1
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 35200245760 (32.78 GB)
DFS Remaining: 35200196608 (32.78 GB)
DFS Used: 49152 (48 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.18.0.4:9866 (p4-dn-1.p4_default)
Hostname: 18e00c58ba78
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 24576 (24 KB)
Non DFS Used: 8204152832 (7.64 GB)
DFS Remaining: 17600098304 (16.39 GB)
DFS Used%: 0.00%
DFS Remaining%: 68.16%
Configured Cache Capacity: 0 (0 B)
C

In [3]:
![ ! -e "hdma-wi-2021.csv" ] && wget "https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv"

In [4]:
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -cp hdma-wi-2021.csv hdfs://boss:9000/single.csv
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -cp hdma-wi-2021.csv hdfs://boss:9000/double.csv

In [5]:
# # # use for clear the files
# !hdfs dfs -rm -f hdfs://boss:9000/single.csv
# !hdfs dfs -rm -f hdfs://boss:9000/double.csv

In [6]:
#q2
!hdfs dfs -du -h hdfs://boss:9000/

166.8 M  333.7 M  hdfs://boss:9000/double.csv
166.8 M  166.8 M  hdfs://boss:9000/single.csv


In [7]:
#q3
import requests
r3 = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS")
r3.json()

{'FileStatus': {'accessTime': 1698264044910,
  'blockSize': 1048576,
  'childrenNum': 0,
  'fileId': 16386,
  'group': 'supergroup',
  'length': 174944099,
  'modificationTime': 1698264052174,
  'owner': 'root',
  'pathSuffix': '',
  'permission': '644',
  'replication': 1,
  'storagePolicy': 0,
  'type': 'FILE'}}

In [8]:
#q4
r4 = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=0&noredirect=true")
r4.json()["Location"]

'http://00e8e0912048:9864/webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&offset=0'

In [9]:
#q5
container_locations = {}
for offset in range(0, r3.json()["FileStatus"]["length"], r3.json()["FileStatus"]["blockSize"]):
    req = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={}&noredirect=true".format(offset))
    output_str = req.json()["Location"]
    container_loc = (output_str.split("/")[2].split(":")[0])
    if container_loc not in container_locations:
        container_locations[container_loc] = 1
    else:
        container_locations[container_loc] += 1

container_locations

{'00e8e0912048': 85, '18e00c58ba78': 82}

In [10]:
import pyarrow as pa
import pyarrow.fs
hdfs = pa.fs.HadoopFileSystem("boss", 9000)

2023-10-25 20:01:13,050 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
#q6
output = None
with hdfs.open_input_file("/single.csv") as file:
    output = file.read_at(nbytes=10, offset=0)
output

b'activity_y'

In [12]:
#q7
import io

target = "Single Family"
count = 0
with hdfs.open_input_file("/single.csv") as file:
    reader = io.TextIOWrapper(io.BufferedReader(file))
    for line in reader:
        if line.find(target) != -1:
            count += 1
count

444874